In [4]:
import pandas as pd
gp = pd.DataFrame.from_dict({
        "residential_territory": 0.25,
        "industrial_territory": 0.12,
        "business_territory": 0.08,
        "recreation_territory": 0.3,
        "transport_territory": 0.1,
        "agricalture_territory": 0.03,
        "special_territory": 0.02,
    },orient='index',columns = ['ratio'])
gp['ratio'] = gp['ratio']/gp['ratio'].sum()
gp

,ratio
residential_territory,0.277778
industrial_territory,0.133333
business_territory,0.088889
recreation_territory,0.333333
transport_territory,0.111111
agricalture_territory,0.033333
special_territory,0.022222


In [2]:
import pandas as pd
from func_zones import gen_plan
fz = pd.DataFrame.from_dict({
        "residential": 0.0,
        "industrial": 0.0,
        "business": 0.0,
        "recreation": 0.0,
        "transport": 0.0,
        "agriculture": 0.0,
        "special": 0.0,
    }, orient='index',columns=['ratio'])

scenarios  =pd.DataFrame.from_dict(gen_plan.func_zone_ratio, orient='index', columns=['ratio'])
for scenario,row1 in scenarios.iterrows():
    ratio1 = row1['ratio']
    
    for func_zone,ratio2 in scenario.zones_ratio.items():
        fz.at[func_zone.name,'ratio'] = fz.at[func_zone.name,'ratio']+ratio1*(ratio2/sum(scenario.zones_ratio.values()))
fz['ratio2'] = fz['ratio']/sum(fz['ratio'])
fz

,ratio,ratio2
residential,0.237517,0.238232
industrial,0.098300,0.098596
business,0.140399,0.140821
recreation,0.235411,0.236120
transport,0.146199,0.146639
agriculture,0.088818,0.089085
special,0.050356,0.050507


In [1]:
import geopandas as gpd
from gen_planner.python.src.genplanner import GenPlanner
from func_zones import func_zones,basic_scenario,gen_plan

res = GenPlanner(gpd.read_file('1.gpkg')).terr2district2zone2block(gen_plan)


ValueError: site2room : [1 2 2 2 4 0] 
 areas:                                           zone_name  ...  site_indeed
3  <func_zones.models.TerritoryZone object at 0x0...  ...            0
1  <func_zones.models.TerritoryZone object at 0x0...  ...            1
4  <func_zones.models.TerritoryZone object at 0x0...  ...            1
0  <func_zones.models.TerritoryZone object at 0x0...  ...            1
2  <func_zones.models.TerritoryZone object at 0x0...  ...            3

[5 rows x 4 columns] 
 point_radius: 0.17

In [ ]:
full = res.area.sum()
res['area'] = res.area
areas  = res.groupby('terr_zone')['area'].sum().reset_index()
areas['percent'] = areas['area']/full
areas

In [ ]:
res.explore(column='func_zone',tiles='CartoDB positron',cmap='Dark2',categorical=True)

In [3]:
from gen_planner.python.src.genplanner import GenPlanner
from func_zones import func_zones,basic_scenario
import geopandas as gpd
res = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).district2zone2block(basic_scenario)

res.explore(column='terr_zone',tiles='CartoDB positron',cmap='Dark2')

In [1]:
res = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).district2zone2block(basic_scenario)
res.explore(column='func_zone',tiles='CartoDB positron',cmap='Dark2')

NameError: name 'GenPlanner' is not defined

In [2]:
import geopandas as gpd
from gen_planner.python.src.genplanner import GenPlanner
from func_zones import func_zones,basic_scenario
res = GenPlanner(gpd.read_file('светогорск.geojson').buffer(0.1)).district2zone2block(basic_scenario)
res.explore(column='func_zone',tiles='CartoDB positron',cmap='Dark2')

C:\Users\user\AppData\Local\Temp\ipykernel_30380\3075661039.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  res = GenPlanner(gpd.read_file('светогорск.geojson').buffer(0.1)).district2zone2block(basic_scenario)
